In [1]:
import os
import time
import requests
import pandas as pd
import numpy as np
from ta import add_all_ta_features
from ta.utils import dropna

class StockDataHandler:
    def __init__(self, tickers, api_key, outputsize="full", cache_dir="cached_data"):
        self.tickers = tickers
        self.api_key = api_key
        self.outputsize = outputsize  # Get long-term data
        self.cache_dir = cache_dir
        os.makedirs(cache_dir, exist_ok=True)
        self.data = {}

    def fetch_data(self):
        """Fetch and cache 10 years of stock data."""
        base_url = "https://www.alphavantage.co/query"

        for ticker in self.tickers:
            cache_file = os.path.join(self.cache_dir, f"{ticker}.csv")

            # ✅ Check cache first
            if os.path.exists(cache_file):
                print(f"📂 Loading cached data for {ticker}...")
                df = pd.read_csv(cache_file, index_col=0, parse_dates=True)
                self.data[ticker] = df
                continue

            print(f"🌐 Fetching 10 years of data for {ticker} from API...")
            params = {
                "function": "TIME_SERIES_DAILY",
                "symbol": ticker,
                "outputsize": self.outputsize,  # "full" gets 20+ years
                "datatype": "json",
                "apikey": self.api_key
            }

            response = requests.get(base_url, params=params)
            data = response.json()

            if "Time Series (Daily)" not in data:
                print(f"❌ Error fetching {ticker}: {data.get('Note', 'No data available.')}")
                continue

            # ✅ Convert API Response to DataFrame
            df = pd.DataFrame.from_dict(data["Time Series (Daily)"], orient="index")
            df.index = pd.to_datetime(df.index)
            df = df.sort_index()

            print(f"🚀 Raw Columns for {ticker}: {df.columns.tolist()}")  # Debugging output BEFORE renaming


            # ✅ **Rename columns properly**
            df = df.rename(columns={
                "1. open": "open", 
                "2. high": "high",
                "3. low": "low", 
                "4. close": "close",  # ✅ Correcting the key error
                "5. volume": "volume"
            }).astype(float)

            print(f"📊 Columns in {ticker}: {df.columns.tolist()}")  # Debugging output


            # ✅ Keep Only the Last **10 Years**
            ten_years_ago = pd.Timestamp.today() - pd.DateOffset(years=10)
            df = df[df.index >= ten_years_ago]

            # ✅ Add Technical Indicators
            df = dropna(df)
            df = add_all_ta_features(df, open="open", high="high", low="low", close="close", volume="volume")

            # ✅ Save to Cache
            df.to_csv(cache_file)
            self.data[ticker] = df

            # 🕒 **Avoid hitting API limit**
            time.sleep(12)

    def get_data(self):
        """Return stock data as a pandas DataFrame."""
        if not self.data:
            raise ValueError("No valid stock data found. Check API key and ticker symbols.")
        
        # 🔍 Debugging: Print column names to check structure
        for ticker, df in self.data.items():
            print(f"🔍 Checking {ticker} columns: {df.columns.tolist()}")
    
        # ✅ Flatten MultiIndex if necessary (for some versions of pandas)
        for ticker in self.data:
            if isinstance(self.data[ticker].columns, pd.MultiIndex):
                print(f"🔧 Flattening MultiIndex for {ticker}")
                self.data[ticker].columns = self.data[ticker].columns.droplevel(0)
    
        # ✅ Ensure correct column names
        for ticker in self.data:
            if "4. close" in self.data[ticker].columns:
                print(f"🔧 Fixing column name for {ticker}")
                self.data[ticker] = self.data[ticker].rename(columns={"4. close": "close"})
        
        # ✅ Convert data dictionary into a DataFrame
        df = pd.concat(self.data, axis=1)
    
        # ✅ Handle missing values
        df.dropna(how="all", inplace=True)  # Drop rows where all tickers have NaN
        
        if df.empty:
            raise ValueError("Stock data is empty after processing. Check cache or API response.")
        
        return df






In [2]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

class XGBoostTrader:
    def __init__(self, df):
        """Initialize with stock data."""
        self.df = df
        self.tickers = [col.split("_")[0] for col in df.columns]  # ✅ Extract tickers
        self.models = {}

    def prepare_data(self, stock_df):
        """Prepare features and target for XGBoost."""
        df = stock_df.copy()
        
        # ✅ Ensure we use the correct column name
        close_col = df.columns[0]  # Should be like 'AAPL_close'
        
        # ✅ Generate features (e.g., simple moving averages)
        df["SMA_10"] = df[close_col].rolling(window=10).mean()
        df["SMA_50"] = df[close_col].rolling(window=50).mean()
        df["SMA_200"] = df[close_col].rolling(window=200).mean()
        
        # ✅ Target: 1 if price increases next day, else 0
        df["target"] = (df[close_col].shift(-1) > df[close_col]).astype(int)
        df.dropna(inplace=True)

        # ✅ Split into training and test sets
        X = df.drop(columns=[close_col, "target"])
        y = df["target"]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

        return X_train, X_test, y_train, y_test

    def train_models(self):
        """Train XGBoost models for each stock."""
        self.models = {}

        for ticker in self.tickers:
            column_name = f"{ticker}_close"
            
            if column_name not in self.df.columns:
                print(f"❌ Skipping {ticker}: Column {column_name} not found.")
                continue
            
            print(f"🚀 Training XGBoost for {ticker}...")
            stock_df = self.df[[column_name]]  # ✅ Use only the correct column
            
            X_train, X_test, y_train, y_test = self.prepare_data(stock_df)
            model = xgb.XGBClassifier(eval_metric="logloss")
            model.fit(X_train, y_train)

            self.models[ticker] = model

            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            print(f"📈 Accuracy for {ticker}: {accuracy:.4f}")

    def predict(self, df):
        predictions = {}
    
        # ✅ Ensure we keep a full DataFrame, not separate per-stock DataFrames
        stock_df = df.copy()
    
        for ticker in self.tickers:
            feature_columns = [f"{ticker}_SMA_10", f"{ticker}_SMA_50", f"{ticker}_SMA_200"]
    
            close_col = f"{ticker}_close"
            if close_col not in stock_df.columns:
                print(f"❌ Warning: Missing {close_col} in DataFrame. Skipping {ticker}.")
                continue
    
            # ✅ Generate missing features if needed
            if not all(col in stock_df.columns for col in feature_columns):
                print(f"🔧 Generating missing features for {ticker}...")
                stock_df.loc[:, f"{ticker}_SMA_10"] = stock_df[close_col].rolling(window=10).mean()
                stock_df.loc[:, f"{ticker}_SMA_50"] = stock_df[close_col].rolling(window=50).mean()
                stock_df.loc[:, f"{ticker}_SMA_200"] = stock_df[close_col].rolling(window=200).mean()
                stock_df.dropna(inplace=True)  # ✅ Remove NaN values
    
            feature_columns = [col for col in feature_columns if col in stock_df.columns]  # ✅ Validate feature presence
            if len(feature_columns) < 3:
                print(f"❌ Warning: Not enough features for {ticker} (Expected 3, Found {len(feature_columns)}). Skipping.")
                continue
    
            if ticker not in self.models:
                print(f"❌ Warning: Model for {ticker} not found. Skipping.")
                continue
    
            # ✅ Use the last row for prediction
            X = stock_df[feature_columns].tail(1).values.reshape(1, -1)
    
            print(f"🔍 Predicting for {ticker}, Feature Shape: {X.shape}")  # Debug print
    
            predictions[ticker] = self.models[ticker].predict(X)[0]
    
        return pd.DataFrame(predictions, index=[stock_df.index[-1]])  # ✅ Convert to DataFrame for consistency









In [3]:
import numpy as np
import pandas as pd
import gymnasium as gym
from gym import spaces

class StockTradingEnv(gym.Env):
    def __init__(self, tickers, stock_data, xgb_trader, initial_cash=10000):
        super(StockTradingEnv, self).__init__()
        self.tickers = tickers
        self.df = stock_data if isinstance(stock_data, pd.DataFrame) else pd.concat(stock_data.values(), axis=1, keys=stock_data.keys())
        self.initial_cash = initial_cash
        self.cash = initial_cash
        self.shares = {ticker: 0 for ticker in self.tickers}
        self.current_step = 0
        self.xgb_trader = xgb_trader  # ✅ Ensure trader is properly passed in

        # ✅ FIX: Use MultiDiscrete instead of Box
        self.action_space = spaces.MultiDiscrete(np.array([3] * len(self.tickers), dtype=np.int32))


        # ✅ Print for debugging
        print(f"✅ Action Space: {self.action_space}")

        # ✅ Ensure observation space is properly formatted
        num_features = len(self.df.columns) + 1 + len(self.tickers)  # Prices, technical indicators + cash + holdings
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(num_features,), dtype=np.float32)

    def reset(self):
        """Resets the environment."""
        self.cash = self.initial_cash
        self.shares = {ticker: 0 for ticker in self.tickers}
        self.current_step = 0
        return self._get_obs(), {}  # ✅ Return observation & info dict

    def step(self, action):
        """✅ Execute an action in the environment."""
        if self.current_step >= len(self.df) - 1:
            return self._get_obs(), 0, True, {}
    
        prices = self.df.iloc[self.current_step]
    
        for i, ticker in enumerate(self.tickers):
            action_value = action[i]
    
            if action_value == 1 and self.cash >= prices[ticker]:  # Buy
                self.shares[ticker] += 1
                self.cash -= prices[ticker]
            elif action_value == 2 and self.shares[ticker] > 0:  # Sell
                self.shares[ticker] -= 1
                self.cash += prices[ticker]
            # Hold (0) does nothing
    
        # ✅ Advance to the next step
        self.current_step += 1
        done = self.current_step >= len(self.df) - 1
    
        # ✅ Calculate reward based on portfolio value
        portfolio_value = self.cash + sum(self.shares[t] * prices[t] for t in self.tickers)
        reward = portfolio_value - self.initial_cash  # Reward = Profit Change
    
        return self._get_obs(), reward, done, {}


    def _get_obs(self):
        """✅ Get the current state as a flat observation array."""
        prices = self.df.iloc[self.current_step].values.flatten().astype(np.float32)
        holdings = np.array([self.shares[t] for t in self.tickers], dtype=np.float32)
    
        # ✅ Ensure cash is a 1D array
        cash_array = np.array([self.cash], dtype=np.float32)
    
        obs = np.concatenate((cash_array, prices, holdings), axis=0)
    
        # ✅ Ensure `xgb_predictions` are properly formatted
        if self.xgb_trader is not None:
            xgb_predictions = np.array(
                [self.xgb_trader[ticker] if ticker in self.xgb_trader else 0 for ticker in self.tickers],
                dtype=np.float32
            ).flatten()
    
            obs = np.concatenate((obs, xgb_predictions), axis=0)
    
        # ✅ Convert explicitly to a NumPy array with shape (n_features,)
        obs = np.asarray(obs, dtype=np.float32).reshape(1, -1).squeeze()
    
        # ✅ Debug output
        print(f"🔍 Final obs.shape: {obs.shape}, obs.dtype: {obs.dtype}, type(obs): {type(obs)}")
    
        return obs







    def seed(self, seed=None):
        """Sets the seed for reproducibility."""
        np.random.seed(seed)

    def render(self):
        """Prints the portfolio and cash state."""
        portfolio_value = self.cash + sum(self.shares[t] * self.df.iloc[self.current_step][t] for t in self.tickers)

        print(f"Step: {self.current_step}")
        print(f"Cash: ${self.cash:.2f}")
        print("Holdings:", {t: self.shares[t] for t in self.tickers})
        print(f"Total Portfolio Value: ${portfolio_value:.2f}")


In [4]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv

# ✅ Load Data & Train XGBoost
tickers = ["AAPL", "GOOGL", "MSFT", "TSLA"]
data_handler = StockDataHandler(tickers, api_key="YOUR_API_KEY", outputsize="full")
data_handler.fetch_data()
df = data_handler.get_data()

df.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in df.columns]
print(f"📊 Updated DataFrame Columns: {df.columns}")


xgb_trader = XGBoostTrader(df)
xgb_trader.train_models()

# ✅ Print the actual column names
print(f"📊 DataFrame Columns Before Prediction: {df.columns}")

# ✅ Train PPO
#xgb_predictions = {ticker: xgb_trader.predict(df[f"{ticker}_close"]) for ticker in tickers}
xgb_predictions = {}
for ticker in tickers:
    column_name = f"{ticker}_close"
    
    # ✅ Debug check
    if column_name not in df.columns:
        print(f"❌ Warning: {column_name} not found in DataFrame. Skipping {ticker}.")
        continue

    # ✅ Ensure we extract a DataFrame (not a Series)
    stock_data = df[[column_name]]  # Keep it as DataFrame

    for ticker, prediction in xgb_predictions.items():
        print(f"🔍 {ticker} prediction type: {type(prediction)}, shape: {np.array(prediction).shape}")

    xgb_predictions = {ticker: float(np.array(prediction).flatten()[0]) for ticker, prediction in xgb_predictions.items()}


env = DummyVecEnv([lambda: StockTradingEnv(tickers, df, xgb_predictions)])
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=20000)
model.save("stock_trading_ppo_xgb")


📂 Loading cached data for AAPL...
📂 Loading cached data for GOOGL...
📂 Loading cached data for MSFT...
📂 Loading cached data for TSLA...
🔍 Checking AAPL columns: ['4. close']
🔍 Checking GOOGL columns: ['4. close']
🔍 Checking MSFT columns: ['4. close']
🔍 Checking TSLA columns: ['4. close']
🔧 Fixing column name for AAPL
🔧 Fixing column name for GOOGL
🔧 Fixing column name for MSFT
🔧 Fixing column name for TSLA
📊 Updated DataFrame Columns: Index(['AAPL_close', 'GOOGL_close', 'MSFT_close', 'TSLA_close'], dtype='object')
🚀 Training XGBoost for AAPL...
📈 Accuracy for AAPL: 0.5140
🚀 Training XGBoost for GOOGL...
📈 Accuracy for GOOGL: 0.4320
🚀 Training XGBoost for MSFT...
📈 Accuracy for MSFT: 0.4687
🚀 Training XGBoost for TSLA...
📈 Accuracy for TSLA: 0.5119
📊 DataFrame Columns Before Prediction: Index(['AAPL_close', 'GOOGL_close', 'MSFT_close', 'TSLA_close'], dtype='object')
✅ Action Space: MultiDiscrete([3 3 3 3])
Using cpu device


AssertionError: The algorithm only supports (<class 'gymnasium.spaces.box.Box'>, <class 'gymnasium.spaces.discrete.Discrete'>, <class 'gymnasium.spaces.multi_discrete.MultiDiscrete'>, <class 'gymnasium.spaces.multi_binary.MultiBinary'>) as action spaces but MultiDiscrete([3 3 3 3]) was provided